<a href="https://colab.research.google.com/github/bpt2017/MachineLearning/blob/master/KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Keras Tuner**

The Keras Tuner is a library that optimize set of hyperparameters for TensorFlow. (1) Model hyperparameters, (2) Algorithm hyperparameters.

### Setup

In [0]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 61kB 1.8MB/s 


### Download and perpare the dataset

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

### Define the model

In [0]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step=32)
  model.add(keras.layers.Dense(units = hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  model.compile(optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  
  return model

### Instantiate the tuner and perform hypertuning

In [0]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [0]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait=True)

In [8]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks=[ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.""")

INFO:tensorflow:Oracle triggered exit
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 352 and the optimal learning rate for the optimizer
is 0.001.


In [9]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4730 - accuracy: 0.8321 - val_loss: 0.4025 - val_accuracy: 0.8575
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3612 - accuracy: 0.8681 - val_loss: 0.3781 - val_accuracy: 0.8620
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3241 - accuracy: 0.8816 - val_loss: 0.3725 - val_accuracy: 0.8671
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2986 - accuracy: 0.8896 - val_loss: 0.3596 - val_accuracy: 0.8718
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2826 - accuracy: 0.8954 - val_loss: 0.3392 - val_accuracy: 0.8790
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2676 - accuracy: 0.8999 - val_loss: 0.3302 - val_accuracy: 0.8812
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2521 - accuracy: 0.9057 - val_loss: 0.3307 - val_accuracy: